In [1]:
%matplotlib qt

import time
import numpy as np
from skimage.filters import gaussian_filter

from tifffile import imread, imsave
import cPickle as pickle

from spimagine import volshow
from spimagine import EllipsoidMesh, Mesh

%reload_ext autoreload
%autoreload 2

from netsurface3d import NetSurf3d

<pyopencl.Device 'HD Graphics 4000' on 'Apple' at 0x1024400>


/Users/jug/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/pyopencl/__init__.py:207: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  "to see more.", CompilerWarning)


could not open /Users/jug/.spimagine
<pyopencl.Device 'HD Graphics 4000' on 'Apple' at 0x1024400>


# Load dataset and show in Spimagine 

In [3]:
filenames_data = ['/Users/jug/Repositories/GIT/BobSeg/MartinMiccai17/final_nuclei/blurred.tif',
                  '/Users/jug/Repositories/GIT/BobSeg/MartinMiccai17/final_nuclei/network_8.tif',
                  '/Users/jug/Repositories/GIT/BobSeg/MartinMiccai17/final_nuclei/isotropic.tif']

# data gradient images        
filenames_grad = ['/Users/jug/Repositories/GIT/BobSeg/MartinMiccai17/final_nuclei/blurred_grad.tif',
                  '/Users/jug/Repositories/GIT/BobSeg/MartinMiccai17/final_nuclei/network_8_grad.tif',
                  '/Users/jug/Repositories/GIT/BobSeg/MartinMiccai17/final_nuclei/isotropic_grad.tif']
        
# watershed segmentation       
filenames_ws = ['/Users/jug/Repositories/GIT/BobSeg/MartinMiccai17/final_nuclei/fiji_output/watershed_blurred.tif',
                '/Users/jug/Repositories/GIT/BobSeg/MartinMiccai17/final_nuclei/fiji_output/watershed_network.tif',
                '/Users/jug/Repositories/GIT/BobSeg/MartinMiccai17/final_nuclei/fiji_output/watershed_iso.tif']

# watershed segmentation (gradients)       
filenames_wsgrad = ['/Users/jug/Repositories/GIT/BobSeg/MartinMiccai17/final_nuclei/fiji_output/watershed_blurred_grad.tif',
                '/Users/jug/Repositories/GIT/BobSeg/MartinMiccai17/final_nuclei/fiji_output/watershed_network_grad.tif',
                '/Users/jug/Repositories/GIT/BobSeg/MartinMiccai17/final_nuclei/fiji_output/watershed_iso_grad.tif']

images_data = [None]*len(filenames_data)
for i in range(len(filenames_data)):
    images_data[i] = imread(filenames_data[i])
    print 'Image dimensions: ', images_data[i].shape

images_grad = [None]*len(filenames_grad)
for i in range(len(filenames_grad)):
    images_grad[i] = imread(filenames_grad[i])
    print 'Image dimensions: ', images_grad[i].shape

images_ws = [None]*len(filenames_ws)
for i in range(len(filenames_ws)):
    images_ws[i] = imread(filenames_ws[i])
    print 'Image dimensions: ', images_ws[i].shape

images_wsgrad = [None]*len(filenames_wsgrad)
for i in range(len(filenames_wsgrad)):
    images_wsgrad[i] = imread(filenames_wsgrad[i])
    print 'Image dimensions: ', images_wsgrad[i].shape

Image dimensions:  (512, 512, 512)
Image dimensions:  (512, 512, 512)
Image dimensions:  (512, 512, 512)
Image dimensions:  (512, 512, 512)
Image dimensions:  (512, 512, 512)
Image dimensions:  (512, 512, 512)
Image dimensions:  (512, 512, 512)
Image dimensions:  (512, 512, 512)
Image dimensions:  (512, 512, 512)
Image dimensions:  (512, 512, 512)
Image dimensions:  (512, 512, 512)
Image dimensions:  (512, 512, 512)


In [5]:
bgframe = 0

In [6]:
volwin = volshow(images_data[bgframe], stackUnits = [1.,1.,1.], raise_window=True)
print 'Dataset now visible in separate Spimagine window...'

Dataset now visible in separate Spimagine window...


# Load vectors sampling unit sphere 

In [7]:
# load pickeled unit sphere sampling
with open('sphere_sampling.pkl','r') as f:
    dictSphereData = pickle.load(f)
    
# sampling parameters
vectors = dictSphereData['points']
neighbors = dictSphereData['neighbors']
neighbors_of = dictSphereData['neighbors_of']
triangles = dictSphereData['triangles']

# Do segmentations using NetSurf3d

**some random cells**

In [26]:
segframe = 1
segdata=images_grad
# segdata=images_wsgrad
K = 30
max_delta_k = 2

cell1_center = (343,297,264)
cell2_center = (337,282,277)
cell3_center = (332,263,296)
cell4_center = (340,252,273)
cell5_center = (339,236,287)


cell1 = NetSurf3d( vectors, triangles, neighbors_of, K=K, max_delta_k=max_delta_k )
cell2 = NetSurf3d( vectors, triangles, neighbors_of, K=K, max_delta_k=max_delta_k )
cell3 = NetSurf3d( vectors, triangles, neighbors_of, K=K, max_delta_k=max_delta_k )
cell4 = NetSurf3d( vectors, triangles, neighbors_of, K=K, max_delta_k=max_delta_k )
cell5 = NetSurf3d( vectors, triangles, neighbors_of, K=K, max_delta_k=max_delta_k )

In [27]:
max_rs = (segdata[0].shape[2] / 15,
          segdata[0].shape[1] / 15,
          segdata[0].shape[0] / 15)

min_rs = (2,2,2)

optimum = cell1.apply_to( segdata[frame], cell1_center, max_rs, min_radii=min_rs )
print 'Optimum energy cell 1: ', optimum
ins, outs = cell1.get_counts()
print 'Nodes in/out: ', ins, outs

optimum = cell2.apply_to( segdata[frame], cell2_center, max_rs ) #, min_radii=min_rs )
print 'Optimum energy cell 2: ', optimum
ins, outs = cell2.get_counts()
print 'Nodes in/out: ', ins, outs

optimum = cell3.apply_to( segdata[frame], cell3_center, max_rs ) #, min_radii=min_rs )
print 'Optimum energy cell 3: ', optimum
ins, outs = cell3.get_counts()
print 'Nodes in/out: ', ins, outs

optimum = cell4.apply_to( segdata[frame], cell4_center, max_rs ) #, min_radii=min_rs )
print 'Optimum energy cell 4: ', optimum
ins, outs = cell4.get_counts()
print 'Nodes in/out: ', ins, outs

optimum = cell5.apply_to( segdata[frame], cell5_center, max_rs ) #, min_radii=min_rs )
print 'Optimum energy cell 5: ', optimum
ins, outs = cell5.get_counts()
print 'Nodes in/out: ', ins, outs

Optimum energy cell 1:  85990.0
Nodes in/out:  6987 8373
Optimum energy cell 2:  84432.0
Nodes in/out:  7994 7366
Optimum energy cell 3:  80665.0
Nodes in/out:  7245 8115
Optimum energy cell 4:  83097.0
Nodes in/out:  7900 7460
Optimum energy cell 5:  86582.0
Nodes in/out:  7252 8108


In [28]:
colors_grey = [(1.-.15*i,1.-.15*i,1.-.15*i) for i in range(6)]
colors_red = [(1.,.2,.2*i) for i in range(6)]
colors_gold = [(1.,.8,.15*i) for i in range(6)]
colors_yellow = [(1.,1.,.9-.15*i) for i in range(6)]
colors_green = [(.45,1.,.4+.1*i) for i in range(6)]
colors_blue = [(.4,1.,1.3+1*i) for i in range(6)]
colors_darkblue = [(.1,.3,1.0-.1*i) for i in range(6)]

In [29]:
#remove all meshes
while len(volwin.glWidget.meshes)>0:
    volwin.glWidget.meshes.pop(0)
volwin.glWidget.refresh()

In [30]:
volwin.glWidget.add_mesh( cell1.create_center_mesh( facecolor=colors_gold[0]) )
volwin.glWidget.add_mesh( cell2.create_center_mesh( facecolor=colors_gold[1]) )
volwin.glWidget.add_mesh( cell3.create_center_mesh( facecolor=colors_gold[2]) )
volwin.glWidget.add_mesh( cell4.create_center_mesh( facecolor=colors_gold[3]) )
volwin.glWidget.add_mesh( cell5.create_center_mesh( facecolor=colors_gold[4]) )

In [31]:
mesh1 = cell1.create_surface_mesh( facecolor=colors_gold[0] )
volwin.glWidget.add_mesh( mesh1 )

mesh2 = cell2.create_surface_mesh( facecolor=colors_gold[1] )
volwin.glWidget.add_mesh( mesh2 )

mesh3 = cell3.create_surface_mesh( facecolor=colors_gold[2] )
volwin.glWidget.add_mesh( mesh3 )

mesh4 = cell4.create_surface_mesh( facecolor=colors_gold[3] )
volwin.glWidget.add_mesh( mesh4 )

mesh5 = cell5.create_surface_mesh( facecolor=colors_gold[4] )
volwin.glWidget.add_mesh( mesh5 )

# Measure Volume

In [416]:
calibration = (1.,1.,5.)

v1 = cell1.get_volume(calibration)
v2 = cell2.get_volume(calibration)
print 'Volume of e cell 1:', v1, ' - thats equivalent to a cube of side length ', v1**(1./3)
print 'Volume of e cell 2:', v2, ' - thats equivalent to a cube of side length ', v2**(1./3)
print 

for i in range(9): #nn):
    v = cells[i].get_volume(calibration)
    print 'Volume of cell',i+1,' is ', v

#idea 2: filter all points between to results that are closer then come value d. compute area of all 
#        triangles defines by those points - touching surface!

Volume of e cell 1: 157157.5  - thats equivalent to a cube of side length  53.964940663
Volume of e cell 2: 149616.666667  - thats equivalent to a cube of side length  53.0876284648

Volume of cell 1  is  50896.6666667
Volume of cell 2  is  67134.1666667
Volume of cell 3  is  128602.5
Volume of cell 4  is  118399.166667
Volume of cell 5  is  93502.5
Volume of cell 6  is  158164.166667
Volume of cell 7  is  103726.666667
Volume of cell 8  is  101182.5
Volume of cell 9  is  48278.3333333


In [153]:
t = volwin.transform.toTransformData()

In [16]:
from spimagine import Quaternion
from spimagine.models.keyframe_model import TransformData

In [17]:
t = TransformData(
        quatRot = Quaternion(0.71936901545,-0.00736075256703,0.694582826718,-0.00295556226069),
        translate= [-0.30246097 ,-0.01346884 ,-0.01174765],
        zoom= 1.37118695139,
        minVal= 1e-06,
        maxVal= 649.866693642,
        gamma=1.0,
        bounds=[ 0.31,  1. ,  -1.  ,  1. ,  -1.,    1.  ],
        isBox=True,
        isIso=False,
        alphaPow=0.72)

In [18]:
from spimagine.models.keyframe_model import TransformData

In [19]:
volwin.transform.fromTransformData(t)